# Whatsapp Chat Analysis

In [ ]:
import pandas as pd

from whatsapp_reader import WhatsappReader

## Extract Data

In [ ]:
# Define path of whatsapp chat txt file
file_path = "data/sample.txt"

In [ ]:
# Call class to export chat as df
whatsapp_reader = WhatsappReader()
chat_df = whatsapp_reader.read_file(file_path)

In [ ]:
chat_df.head(10)